In [ ]:
!pip install opencv-python
!pip install mediapipe

In [4]:
import cv2

cap = cv2.VideoCapture(0)
import mediapipe as mp
mp_drawing = mp.solutions.drawing_utils
mp_drawing_styles = mp.solutions.drawing_styles
mp_hands = mp.solutions.hands
import numpy as np

In [2]:
def dist(a,b):
    return np.sqrt(np.square(a.x-b.x)+np.square(a.y-b.y)+np.square(a.z-b.z))

In [5]:
thumb_tip = 4
index_finger_tip = 8
# Read logo
logo = cv2.imread('spark.png', cv2.IMREAD_UNCHANGED)

size = 100

# For webcam input:
cap = cv2.VideoCapture(0)
with mp_hands.Hands(
    model_complexity=0,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5) as hands:
  while cap.isOpened():
    success, image = cap.read()
    #print(image.shape)
    if not success:
      print("Ignoring empty camera frame.")
      # If loading a video, use 'break' instead of 'continue'.
      continue

    # To improve performance, optionally mark the image as not writeable to
    # pass by reference.
    image.flags.writeable = False
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    results = hands.process(image)

    # Draw the hand annotations on the image.
    image.flags.writeable = True
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    if results.multi_hand_landmarks and len(results.multi_hand_landmarks) > 1 :
      #thumb_coords = results.multi_hand_landmarks[0].landmark[thumb_tip]
      index_finger_coords_0 = results.multi_hand_landmarks[0].landmark[index_finger_tip]
      index_finger_coords_1 = results.multi_hand_landmarks[1].landmark[index_finger_tip]
      #print(dist(thumb_coords,index_finger_coords))
      if dist(index_finger_coords_0,index_finger_coords_1) < 0.2:
        # position of logo on video
        top_left = [index_finger_coords_0.x*image.shape[1], index_finger_coords_0.y*image.shape[0]] # the top-left corner of your logo goes here
        tx = int(top_left[0]) # less typing later
        ty = int(top_left[1] - size / 2)

        # crop of logo
        left = 0
        right = size
        top = 0 # y = 0 is the top of the image
        bottom = size

        # calculate width and height of logo crop
        width = right - left
        height = bottom - top


        # get crops of background and logo
        print(tx,ty)
        background_slice = image[ty:ty+height, tx:tx+width]; 
        logo_slice = logo[0:background_slice.shape[1],0:background_slice.shape[0],:]; 

        # since we don't change our crop, we could do the logo slice outside the loop
        # but I'll keep it here for clarity

        result = np.zeros((background_slice.shape[1], background_slice.shape[0], 3), np.uint8)

        alpha = logo_slice[:, :, 3] / 255.0
        result[:, :, 0] = (1. - alpha) * background_slice[:, :, 0] + alpha * logo_slice[:, :, 0]
        result[:, :, 1] = (1. - alpha) * background_slice[:, :, 0] + alpha * logo_slice[:, :, 1]
        result[:, :, 2] = (1. - alpha) * background_slice[:, :, 0] + alpha * logo_slice[:, :, 2]

        # add slice of logo onto slice of background
        #added_image = cv2.addWeighted(logo_slice,1,background_slice,0,0)
        image[ty:ty+height, tx:tx+width] = result

      for hand_landmarks in results.multi_hand_landmarks:
        
        mp_drawing.draw_landmarks(
            image,
            hand_landmarks,
            mp_hands.HAND_CONNECTIONS,
            mp_drawing_styles.get_default_hand_landmarks_style(),
            mp_drawing_styles.get_default_hand_connections_style())
    # Flip the image horizontally for a selfie-view display.
    cv2.imshow('MediaPipe Hands', image)


    if cv2.waitKey(5) & 0xFF == 27:
      break
      
cap.release()

357 109
370 111
373 111
376 114
376 114
382 114
381 114
385 115
385 115
389 113
387 111
390 109
391 111
393 113
393 111
393 113
393 112
393 113
393 114
389 115
388 118
384 125
379 133
378 129
375 110
373 108
376 104
376 106
373 102
385 128
387 118
388 148
385 150
383 152
381 155
378 158
375 162
372 162
371 163
368 163
369 165
365 165
368 162
374 149
374 150
377 144
379 141
379 138
380 134
386 131
385 131
384 127
387 131
389 130
389 133
388 141
390 142
389 142
392 143
390 141
393 143
390 144
391 145
388 148
390 146
389 147
393 144
396 147
398 146
398 148
398 148
402 147
406 152
414 154
446 111
450 107
458 108
455 104
458 103
458 102
460 102
465 102
468 99
472 97
472 94
477 93
479 93
480 93
475 107
479 126
474 132
474 136
476 133
473 136
478 133
477 132
476 135
476 137
477 135
473 140
475 139
472 140
477 141
475 141
476 142
477 142
481 142
478 157
485 136
483 149
484 150
485 155
401 143
396 136
397 134
399 133
402 135
399 136
402 137
403 137
403 134
404 131
404 131
407 128
406 132
406 13

KeyboardInterrupt: 